In [6]:
# Created on Jan 29, 2020
# Autor: Alex Mottus
import pandas as pd # library to process data as dataframes

df_boroughes=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# Read in the first dataframe - read_html creates multiple dataframs
df_borough=df_boroughes[0]
# Remove records which have a 'Borough' value of 'Not Assigned'
df_borough.drop(df_borough[df_borough['Borough']=='Not assigned'].index, inplace=True)
# Loop through each row in the dataframe checking for Neighbourhoods with the value 'Not assigned'. \
# If one is found move over the value in the Borough cell
for index, row in df_borough.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']= row['Borough']
# Identify all records with the same Postal Code and Borough: merger the Neighbourhood values in a comma delimited format
df_b=df_borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
# Rename the postal code column
df_b.rename(columns={'Postcode':'Postalcode'}, inplace=True)

#print(df_b.shape)

# Read in Geospactial data by Postal Code
df_data = pd.read_csv('http://cocl.us/Geospatial_data')
#print(df_data.head())
# My Code resumes here: Merge Dataframes based on the matching postal code fields
df_final=df_b.merge(df_data,how='inner', left_on='Postalcode', right_on='Postal Code')
# Remove extra postal code column
del df_final['Postal Code']
# Display top 10 Records

#df_final.head(10)

# Map Data
#!pip install folium
import folium # plotting library
from sklearn.cluster import KMeans

latitude=43.6532
longitude=-79.3832
toronto_map=folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred on Toronto
tor_data = df_final[df_final['Borough'].str.contains('Toronto')].reset_index(drop=True)
#df[df['model'].str.contains('ac')]

# set number of clusters
kclusters = 5

#manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_final)
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#kmeans.labels_[0:10] 

for lat, lng, label in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['Borough']):
    folium.Marker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(toronto_map)

#tor_data.head()
toronto_map
